# Setting up demo service

First download data from digitaliseringskataloget.dk. In this case from https://digitaliseringskataloget.dk/integration/sf0101 and use the "Download dokumentation". 

Unzip the file and save it to this folder. Secondly there is a zip file within the zip file that we also unzip.

The result of this is included in this template and saved in the folder "SF0101+-+Dokumentationspakke+til+DemoService_1.0".

The template for the headers are already in the "files_with_sts_service_templates" folder so all we need to do is create the body of the request.

We use the python library zeep to create the request body. The library is installed in the requirements.txt file.



### Show contents of WSDL file

First we demonstrate how to use Python how to read the WSDL file and show the contents of the file.



In [29]:
import os

path_to_wsdl_file = "../SF0101+-+Dokumentationspakke+til+DemoService_1.0/SF0101 Teknisk Spec/wsdl/token/DemoService.wsdl"

print(os.popen(f"python -mzeep '{path_to_wsdl_file}'").read())


Prefixes:
     xsd: http://www.w3.org/2001/XMLSchema
     ns0: http://serviceplatformen.dk/xml/schemas/ServiceplatformFault/1/
     ns1: http://serviceplatformen.dk/xml/wsdl/soap11/SP/Demo/1/
     ns2: http://serviceplatformen.dk/xml/schemas/CallContext/1/
     ns3: http://serviceplatformen.dk/xml/schemas/AuthorityContext/1/
     ns4: http://serviceplatformen.dk/xml/schemas/InvocationContext/1/

Global elements:
     ns3:AuthorityContext(ns3:AuthorityContextType)
     ns2:AccountingInfo(ns2:AccountingInfoType)
     ns2:CallContext(ns2:CallContextType)
     ns2:CallersServiceCallIdentifier(ns2:CallersServiceCallIdentifierType)
     ns2:OnBehalfOfUser(ns2:OnBehalfOfUserType)
     ns4:AccountingInfo(ns4:AccountingInfoType)
     ns4:CallersServiceCallIdentifier(ns4:CallersServiceCallIdentifierType)
     ns4:InvocationContext(ns4:InvocationContextType)
     ns4:OnBehalfOfUser(ns4:OnBehalfOfUserType)
     ns4:ServiceAgreementUUID(ns4:ServiceAgreenentUUIDtype)
     ns4:ServiceUUID(ns4:Servic

### Create body

The next step is illustrating the creation of the body of the request. We use the python library zeep to create the request body. The library is installed in the requirements.txt file.

The body is created by using the zeep library to create a python object that is then converted to a string. The string is then used as the body of the request.

In other cases it is slightly more involved to create the body of the request. 

There are more examples of how to create the body of the request in the SoapClases.py file under the "create_body_for_xml_service" function.

In [31]:
 # Use zeep to create body
import zeep
from lxml import etree


client = zeep.Client(wsdl=f"{path_to_wsdl_file}")

# Define cvr of municipality
service_operation = "callDemoService"
municipality_cvr=29189846

# create authority context using zeep
soap_body = client.create_message(
                    client.service,
                    "callDemoService",
                    AuthorityContext={
                        "MunicipalityCVR": f"{municipality_cvr}"},
                    messageString="'%%MESSAGE_STRING%%'",
                )
                
# extract content of body which is the third tagged element
for index, child in enumerate(soap_body.iter()):
    if index == 2:
        content_of_body_tag = child.tag
        content_of_body = soap_body.findall(f".//{content_of_body_tag}")

# convert to string in correct format
content_of_body_str = etree.tostring(content_of_body[0], encoding="unicode")

# add correct start and end body with random id. The "_%%ID6%%" is a placeholder for the actual id that will be inserted later
body_start = '<soap:Body xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" wsu:Id="_%%ID6%%">'
body_end = "</soap:Body>"

# combine all parts to create full body
full_body = body_start + content_of_body_str + body_end

# write body to file
with open(
    f"./files_with_body_templates/body_{service_operation}.xml",
    "w",
    encoding="utf8",
) as f:
    f.write(full_body)

### Create sample request

Finally we demonstrate how to create a sample request that can be used to test the service.

We use the Python library requests to create the request. 

Furthermore we also define the necessary environmental variables that are needed to access the service.

In [1]:
import os 

# setup environmental variables
os.environ["CERTIFICATE_NAME"] =  'sp_devtest4_demoklient_sf0101_1.pfx'
os.environ["CERTIFICATE_PASSWORD"] = '1kKUWZ,91Zg1'
os.environ["MUNICIPALITY_CVR"] = '29189846'
os.environ['TEST_OR_PROD'] = 'test'

# Load libraries AFTER setting environmental variables
from calling_endpoints import all_endpoints


# Choose service
service_endpoint = "DemoService"

# Choose text
text = "Hej"

# initialize sts services
sts_services_all = all_endpoints()

result = sts_services_all.call_endpoint(service_endpoint, text)

print(result)


b'<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"><soap:Header><sbf:Framework xmlns:ns1="urn:liberty:sb:profile" xmlns:sbf="urn:liberty:sb:2006-08" xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" ns1:profile="urn:liberty:sb:profile:basic" version="2.0" wsu:Id="_218C4F2D279989795916957095403954751666"/><Action xmlns="http://www.w3.org/2005/08/addressing" xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" wsu:Id="_218C4F2D279989795916957095403954751664">http://serviceplatformen.dk/xml/wsdl/soap11/SP/Demo/1/DemoPortType/callDemoServiceResponse</Action><MessageID xmlns="http://www.w3.org/2005/08/addressing" xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" wsu:Id="_218C4F2D279989795916957095403954751662">urn:uuid:f6a5cd89-8450-4a38-9fe4-daffc944b736</MessageID><To xmlns="http://www.w3.org/2005/08/addressing" xmlns:wsu="http://docs.oas

In [2]:
## Prettify XML

import xml.dom.minidom

# assuming `result` contains the XML output
xml_string = result.decode('utf-8')

# parse the XML string
dom = xml.dom.minidom.parseString(xml_string)

# prettify the XML string
pretty_xml_string = dom.toprettyxml()

# print the prettified XML string
print(pretty_xml_string)


<?xml version="1.0" ?>
<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
	<soap:Header>
		<sbf:Framework xmlns:ns1="urn:liberty:sb:profile" xmlns:sbf="urn:liberty:sb:2006-08" xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" ns1:profile="urn:liberty:sb:profile:basic" version="2.0" wsu:Id="_218C4F2D279989795916957095403954751666"/>
		<Action xmlns="http://www.w3.org/2005/08/addressing" xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" wsu:Id="_218C4F2D279989795916957095403954751664">http://serviceplatformen.dk/xml/wsdl/soap11/SP/Demo/1/DemoPortType/callDemoServiceResponse</Action>
		<MessageID xmlns="http://www.w3.org/2005/08/addressing" xmlns:wsu="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd" wsu:Id="_218C4F2D279989795916957095403954751662">urn:uuid:f6a5cd89-8450-4a38-9fe4-daffc944b736</MessageID>
		<To xmlns="http://www.w3.org/2005/08/add